In [9]:
import os

import pandas as pd
import numpy as np

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

from flask import Flask, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)


In [10]:

app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///db/bk.db"
db = SQLAlchemy(app)

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(db.engine, reflect=True)

# Save references to each table
Samples = Base.classes.link

stmt = db.session.query(Samples).statement
df = pd.read_sql_query(stmt, db.session.bind)


C:\Users\andre\AppData\Local\Continuum\anaconda3\lib\site-packages\flask_sqlalchemy\__init__.py:794: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [12]:
@app.route("/")
def index():
    """Return the homepage."""
    return render_template("index.html")


@app.route("/samples")
def samples():
    """Return `Transaction` and `Item`."""
    stmt = db.session.query(Samples).statement
    df = pd.read_sql_query(stmt, db.session.bind)

    # Filter the data based on the sample number and
    # only keep rows with values above 1
    sample_data = df
    # Format the data to send as json
    data = {
        "item": sample_data.Item.tolist(),
        "date": sample_data.Date.tolist(),
        "time": sample_data.Time.tolist(),
        "transaction": sample_data.Transection.tolist()
    }
    return jsonify(data)


if __name__ == "__main__":
    app.run()

RuntimeError: Working outside of application context.

This typically means that you attempted to use functionality that needed
to interface with the current application object in some way. To solve
this, set up an application context with app.app_context().  See the
documentation for more information.